In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
import sys
sys.path.append('/Users/bengikoseoglu/anaconda2/pkgs')
sys.path.append('/Users/bengikoseoglu/anaconda2/pckgs')
sys.path.append('/Users/bengikoseoglu')
sys.path.append('/Library/Python/2.7/site-packages/')
sys.path.append('/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/nltk')
sys.path.append('/Users/bengikoseoglu/nltk_data')
sys.path

['',
 '/Users/bengikoseoglu/anaconda2/lib/python27.zip',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-darwin',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-tk',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-old',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-dynload',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/aeosa',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/Users/bengikoseoglu/.ipython',
 '/Users/bengikoseoglu/anaconda2/pkgs',
 '/Users/bengikoseoglu/anaconda2/pckgs',
 '/Users/bengikoseoglu',
 '/Library/Python/2.7/site-packages/',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/nltk',
 '/Users/bengikoseoglu/nltk_data']

In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import *
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [4]:
#find file i
df=pd.read_excel('cameras_400_3_nocamera.xlsx')

In [5]:
import os
os.chdir('/Users/bengikoseoglu/Documents/Masters/Semester2/TeamProject/Product_Classification/Adding_htmls')

In [6]:
import os
array_list=os.listdir(os.getcwd())

In [7]:
len(array_list)

23489

In [8]:
array_list[0]

'newegg-14e1a28184ebe70e2864f414e5f7d922.html'

In [9]:
df=df.rename(columns={'Leaf Node':'Leaf_Node'})

In [10]:
q = """
SELECT DISTINCT *, (CASE WHEN Leaf_Node IN ('360 Degree Cameras','Common Use Video Cameras','Non 360 Degree Cameras',
'Professional Video Cameras','Drone') THEN 'Cam Corders' WHEN Leaf_Node IN ('Not Camera') THEN 'Not Camera' ELSE 'Photo Devices' END) AS parent_node_1, 
(CASE WHEN Leaf_Node IN ('360 Degree Cameras','Non 360 Degree Cameras','Drone') THEN 'Action Cameras' 
WHEN Leaf_Node IN ('Bridge Cameras','Digital Compact Cameras','Instant') THEN 'Point & Shoot Cameras' 
WHEN Leaf_Node IN ('Common Use Video Cameras','Professional Video Cameras') THEN 'Traditional Video Cameras' 
WHEN Leaf_Node IN ('DSLR') THEN 'DSLR' 
WHEN Leaf_Node IN ('Not Camera') THEN 'Not Camera'
WHEN Leaf_Node IN ('Full Frame Mirrorless Camera','NonFullFrameMirrorlessCamera') THEN 'Mirrorless Cameras' 
ELSE 'NO CATEGORY'
END) AS parent_node_2
FROM df
"""
df = pysqldf(q)

In [11]:
q = """
SELECT DISTINCT parent_node_1,parent_node_2,Leaf_Node, COUNT(ID)
FROM df
GROUP BY parent_node_1,parent_node_2,Leaf_Node
"""
a = pysqldf(q)
a

,parent_node_1,parent_node_2,Leaf_Node,COUNT(ID)
0,Cam Corders,Action Cameras,360 Degree Cameras,21
1,Cam Corders,Action Cameras,Drone,15
2,Cam Corders,Action Cameras,Non 360 Degree Cameras,22
3,Cam Corders,Traditional Video Cameras,Common Use Video Cameras,21
4,Cam Corders,Traditional Video Cameras,Professional Video Cameras,32
5,Not Camera,Not Camera,Not Camera,19
6,Photo Devices,DSLR,DSLR,72
7,Photo Devices,Mirrorless Cameras,Full Frame Mirrorless Camera,31
8,Photo Devices,Mirrorless Cameras,NonFullFrameMirrorlessCamera,50
9,Photo Devices,Point & Shoot Cameras,Bridge Cameras,56


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=df['name']
vectorizer = CountVectorizer()
matrix=vectorizer.fit_transform(corpus).todense()
vocab=vectorizer.vocabulary_ 
print(matrix.shape)
print(len(vocab))

(416, 875)
875


In [13]:
bags_words=pd.DataFrame(matrix, columns=vocab)
bags_words['ID']=df['ID']
bags_words['Parent_node_1']=df['parent_node_1']
bags_words['Parent_node_2']=df['parent_node_2']
bags_words['Leaf_Node']=df['Leaf_Node']

In [14]:
bags_words.head() #this gets the name

,cameras,oss,yashica,gz,yellow,32mm,012,gb,go,z212,...,hx400,karma,fz43,deluxe,cpt,dvr,ID,Parent_node_1,Parent_node_2,Leaf_Node
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,jessops-f2dd233dd6100c900432fd2902df8cde,Photo Devices,Point & Shoot Cameras,Bridge Cameras
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,jessops-ff2be7ba459709194635812c728a17fd,Photo Devices,DSLR,DSLR
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,jessops-acfa4e6fdc0962be2da5bc1ed8d44357,Photo Devices,DSLR,DSLR
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,jessops-cabb446087d8428bfa741078869439b8,Photo Devices,DSLR,DSLR
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,jessops-8c6e0672aa9297ee8ff4d95fa5e2edc0,Photo Devices,Mirrorless Cameras,Full Frame Mirrorless Camera


In [15]:
df_model=df[['ID','priceCurrency','price','brand']]
brand_dummies=pd.get_dummies(df_model['brand'])
pricecur_dummies=pd.get_dummies(df_model['priceCurrency'])
df_model=pd.concat([df_model,brand_dummies,pricecur_dummies],axis=1) #pd.concat([df1, df4], axis=1
#bags=bags_words.drop(['prediction_level1'],axis=1)
model2=df_model.merge(bags_words,on='ID',how='inner')
model2=model2.drop(['priceCurrency','brand'],axis=1)
a=model2['price'].str.split(',', 1, expand=True)
a=a.drop([206])
a=a.astype(float)
model2['price']=a[0]
median(model2['price'].dropna())
# filling missing values with median
model2['price']=model2['price'].fillna(548)

In [16]:
model2.head()

,ID,price,360fly_x,6Ave,Air Hogs,AmazingForLess,Axess Products,Bell+Howell,BlackMagic,Blackmagic,...,eos,hx400,karma,fz43,deluxe,cpt,dvr,Parent_node_1,Parent_node_2,Leaf_Node
0,jessops-f2dd233dd6100c900432fd2902df8cde,479.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Photo Devices,Point & Shoot Cameras,Bridge Cameras
1,jessops-ff2be7ba459709194635812c728a17fd,539.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Photo Devices,DSLR,DSLR
2,jessops-acfa4e6fdc0962be2da5bc1ed8d44357,3249.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Photo Devices,DSLR,DSLR
3,jessops-cabb446087d8428bfa741078869439b8,1349.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Photo Devices,DSLR,DSLR
4,jessops-8c6e0672aa9297ee8ff4d95fa5e2edc0,449.0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,Photo Devices,Mirrorless Cameras,Full Frame Mirrorless Camera


In [17]:
model2.shape

(416, 957)

### Adding html parsing

In [18]:
import codecs
import unicodedata
i=0
keys=model2['ID'].unique()
idd=[]
n=len(keys)
all_html=[]
err=0
while(i<n):
    thing=(keys[i].encode('ascii','ignore'))+'.html'
    try:
        f=codecs.open(thing, 'r')
        text1=f.read()
        cleantext = BeautifulSoup(text1, "lxml").text
        as_str=cleantext.encode('ascii','ignore')
        clean=as_str.translate(None, '\r\n\t\,)};{$.|:(#"[]/><+=')
        all_html.append(clean)
        idd.append(keys[i].encode('ascii','ignore'))
    except Exception: 
        err=err+1
    i=i+1

In [19]:
len(all_html)

335

In [20]:
notin_id=df[~df['ID'].isin(idd)] #we have some unmatched cameras

In [21]:
notin_id=notin_id['ID'].reset_index()

In [22]:
store=[]
k=0
n1=notin_id.shape[0]
while(k<n1):
    store.append(notin_id['ID'][k][0:5])
    k=k+1

In [23]:
np.unique(store)

array([u'argos', u'clift', u'curry', u'jesso', u'neweg', u'walma'],
      dtype='<U5')

In [24]:
df.head()

,ID,name,Leaf_Node,priceCurrency,price,brand,itemCondition,availability,Description,image,...,gtin13,gtin14,gtin8,ratingValue,bestRating,reviewCount,Unnamed: 17,Good Examples,parent_node_1,parent_node_2
0,jessops-f2dd233dd6100c900432fd2902df8cde,Nikon Coolpix P900 Digital Camera,Bridge Cameras,GBP,479,Nikon,None,True,None,https://i.jessops.com/images/home/homepage/100...,...,NaN,None,2018-09-08 00:00:00.000000,10,162.0,NaN,None,None,Photo Devices,Point & Shoot Cameras
1,jessops-ff2be7ba459709194635812c728a17fd,Canon EOS 750D Digital SLR with 18-55mm IS STM...,DSLR,GBP,539,Canon,None,True,None,https://i.jessops.com/images/home/homepage/100...,...,NaN,None,2018-02-09 00:00:00.000000,10,361.0,NaN,None,None,Photo Devices,DSLR
2,jessops-acfa4e6fdc0962be2da5bc1ed8d44357,Canon EOS 5D Mark IV Digital SLR Body,DSLR,GBP,3249,Canon,None,True,None,https://i.jessops.com/images/home/homepage/100...,...,NaN,None,None,2018-07-09 00:00:00,10.0,25.0,None,None,Photo Devices,DSLR
3,jessops-cabb446087d8428bfa741078869439b8,Canon EOS 7D Mark II Digital SLR Body,DSLR,GBP,1349,Canon,None,True,None,https://i.jessops.com/images/home/homepage/100...,...,NaN,None,None,2018-04-09 00:00:00,10.0,268.0,None,None,Photo Devices,DSLR
4,jessops-8c6e0672aa9297ee8ff4d95fa5e2edc0,Sony A6000 Mirrorless Camera in Black with 16-...,Full Frame Mirrorless Camera,GBP,449,Sony,None,True,None,https://i.jessops.com/images/home/homepage/100...,...,NaN,None,None,9.0,10.0,107.0,None,None,Photo Devices,Mirrorless Cameras


Following these steps: https://www.kdnuggets.com/2018/03/text-data-preprocessing-walkthrough-python.html

1) Noise Removal = removing special characters, that is still present in the dataset

In [25]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [26]:
def denoise_text(text):
    text = remove_between_square_brackets(text)
    return text

In [27]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

In [28]:
import re, string, unicodedata
import contractions
all_html2=[]
for i in range(0, len(all_html)):
    sample = denoise_text(all_html[i])
    sample=replace_contractions(sample)
    all_html2.append(sample)

2) Tokenazation = seperate by the spaces

In [29]:
#words=normalize(map(unicode,all_html2[0]))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
matrix=vectorizer.fit_transform(all_html2).todense()
vocab=vectorizer.vocabulary_ 

In [30]:
vocab #how many times it appears in the text overall

{u'stockclick': 71487,
 u'11700saving3924': 2461,
 u'elnamesub0': 38218,
 u'possiblelive': 60942,
 u'elementsviewfinder': 38057,
 u'380050315231_12e4c5a89d8ac7cfe19e1b6cf7d90a6d9jpegurlckpnikon': 11746,
 u'odnbgffffffzoomhttpsi5walmartimagescomasr28602553': 57116,
 u'circuitry': 31639,
 u'functionfuncvar': 42236,
 u'hanging': 43955,
 u'batterycapacity1480': 25151,
 u'fiturn': 40855,
 u'tudicque': 74747,
 u'u003cu002fliu003eu003cliu003erechargeable': 75663,
 u'item9siafsw6wz4713': 47938,
 u'2f40950575window_wmlrelm': 9955,
 u'infowebsitehttptianxundaqqdcwcomsupport': 47235,
 u'functioncreative': 42207,
 u'cameras3944_133277_1231331_12313353944_133277_1231331_1231335idfuji': 29210,
 u'travelingtextthis': 74512,
 u'6a604dee': 16804,
 u'price865earn': 61464,
 u'sr6818': 70810,
 u'750macradle': 17310,
 u'2359kdotsimaging': 8512,
 u'counter_title1classlistremovehidden': 33779,
 u'footage158': 41341,
 u'autosuperior': 23969,
 u'76af1497': 17405,
 u'sewingsewing1334134_1094704uid4c3wtiq9catego

In [31]:
len(vocab)

80154

In [32]:
len(vocab.keys())

80154

3) Normalization = Normalizing text can mean performing a number of tasks, but for our framework we will approach normalization in 3 distinct steps: 
(1) stemming: Stemming is the process of eliminating affixes (suffixed, prefixes, infixes, circumfixes) from a word in order to obtain a word stem. 
(2) lemmatization: Lemmatization is related to stemming, differing in that lemmatization is able to capture canonical forms based on a word's lemma.
(3) everything else: set all characters to lowercase, remove punctuation, emove default stop words ( "the," "and," and "a,")

In [33]:
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words



def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

In [34]:
vocab_ori=vocab

In [35]:
words=normalize(map(unicode,vocab))

In [36]:
len(vocab.keys())

80154

In [37]:
len(words)

80028

In [38]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

stems, lemmas = stem_and_lemmatize(words)

In [39]:
lemmas

[u'stockclick',
 u'11700saving3924',
 u'elnamesub0',
 u'possiblelive',
 u'elementsviewfinder',
 u'380050315231_12e4c5a89d8ac7cfe19e1b6cf7d90a6d9jpegurlckpnikon',
 u'odnbgffffffzoomhttpsi5walmartimagescomasr28602553',
 u'circuitry',
 u'functionfuncvar',
 u'hang',
 u'batterycapacity1480',
 u'fiturn',
 u'tudicque',
 u'u003cu002fliu003eu003cliu003erechargeable',
 u'item9siafsw6wz4713',
 u'2f40950575window_wmlrelm',
 u'infowebsitehttptianxundaqqdcwcomsupport',
 u'functioncreative',
 u'cameras3944_133277_1231331_12313353944_133277_1231331_1231335idfuji',
 u'travelingtextthis',
 u'6a604dee',
 u'price865earn',
 u'sr6818',
 u'750macradle',
 u'2359kdotsimaging',
 u'counter_title1classlistremovehidden',
 u'footage158',
 u'autosuperior',
 u'76af1497',
 u'sewingsewing1334134_1094704uid4c3wtiq9categoryuidlhn',
 u'elfindshiptimertexthtmlorder',
 u'serviceclickthroughtypeurlvaluecpfood',
 u'qualitytotalreviewcount61standalonelinku002fsiteu002freviewsu002fsony',
 u'stars114',
 u'wednesday',
 u'fz2500th

In [40]:
len(lemmas)

80028

4) Eliminating words that have more letters than 45, because the longest word in vocablary is  pneumonoultramicroscopicsilicovolcanoconiosis and it has 45 characters

In [41]:
n=len(lemmas)
vocab2=[]
for i in range(0, n):
    if(len(lemmas[i])<46):
        vocab2.append(lemmas[i])

In [42]:
len(vocab2)

66719

5) TFIDF

Inspired by https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

- Start by eliminating words from your main data and join with your data

In [43]:
# This is without eliminating anything just tokenizing
bags_words=pd.DataFrame(matrix, columns=vocab)
bags_words['ID']=idd

In [44]:
bags_words.shape 

(335, 80155)

In [45]:
bags_words.head()

,stockclick,11700saving3924,elnamesub0,possiblelive,elementsviewfinder,380050315231_12e4c5a89d8ac7cfe19e1b6cf7d90a6d9jpegurlckpnikon,odnbgffffffzoomhttpsi5walmartimagescomasr28602553,circuitry,functionfuncvar,hanging,...,windowsmac,zoom35,35mm,whiteu002f5623303descriptionbest,cfd,expands,lensesnamepentax,id4880600,approxviewfinder,ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-13fb43ddbd8d9cef68833c1b9dcc8030
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1a4c07203f6dfa7a994b27a7f8dd8d0a
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1e1dc143119f5e0c4deaa9d7ac3f9583
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1e7969c673a012084c9f9bbe0e9794c0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-239b2bed55930613da27e4c3f077b94e


In [46]:
vocab2.append('ID')

In [47]:
df1 = pd.DataFrame(bags_words, columns=vocab2) #take the columns that we have selected previously

In [48]:
df1.shape

(335, 66720)

In [49]:
df1.head()

,stockclick,11700saving3924,elnamesub0,possiblelive,elementsviewfinder,circuitry,functionfuncvar,hang,batterycapacity1480,fiturn,...,windowsmac,zoom35,35mm,whiteu002f5623303descriptionbest,cfd,expand,lensesnamepentax,id4880600,approxviewfinder,ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-13fb43ddbd8d9cef68833c1b9dcc8030
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1a4c07203f6dfa7a994b27a7f8dd8d0a
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1e1dc143119f5e0c4deaa9d7ac3f9583
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-1e7969c673a012084c9f9bbe0e9794c0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,bestbuy-239b2bed55930613da27e4c3f077b94e


In [50]:
#now merge it
model_html=model2.merge(df1, on='ID') #model2 have extracted features, df1 have html annotated.

In [51]:
model_html.shape

(335, 67676)

- Drop missing values

In [52]:
model_html=model_html.dropna(axis=1, how='any') #DROPPING NA values
model_html.shape

(335, 67201)

- We have some duplicating columns (most likely due to merging, we have to find these columns make them one column and assign 1 if they are presernt)

In [53]:
a_un=np.unique(model_html.columns)
len(a_un) #number of unique columns (so we have 64780-62701 number of multiplying columns)

65294

In [54]:
#This finds duplicated columns
c=model_html.columns
b=list(model_html.columns.duplicated())
dupl=[]
n=len(b)
i=0
while(i<n):
    if(b[i]==True):
        dupl.append(c[i])
    i=i+1

In [55]:
print(len(dupl))
print(dupl[0:5]) 

1907
[u'watch', u'initiate', u'research', u'pair', u'machine']


In [56]:
dupll=model_html[dupl]
dupll=dupll.groupby(dupll.columns, axis=1).sum() #duplicated ones
dupll.shape

(335, 1019)

In [57]:
df2_nondup = model_html.drop(dupl, axis=1) #without duplicated
df2_nondup.shape

(335, 64275)

In [58]:
df2_all=pd.concat([df2_nondup, dupll ], axis=1)
df2_all.shape

(335, 65294)

In [59]:
idd=df2_all['ID']
Parent_node_1=df2_all['Parent_node_1']
Parent_node_2=df2_all['Parent_node_2']
leaf_node=df2_all['Leaf_Node']#'ID','Parent_node_1','Parent_node_2','Leaf_Node
prince=df2_all['price_x']
df2_all_d=df2_all.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','price_x'],axis=1)

In [60]:
df2_all_d.head()

,360fly_x,6Ave,Air Hogs,AmazingForLess,Axess Products,Bell+Howell,BlackMagic,Blackmagic,Blackmagic Design,CANON,...,withstand,wonder,work,worry,wrap,write,yaw,yield,zone,zoom_y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Bags of words to TFIDF vector

In [61]:
df2_all_n=df2_all_d.astype(int)

In [62]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf2 = tfidf_transformer.fit_transform(df2_all_n)
X_train_tfidf2.shape

(335, 65289)

In [63]:
X_train_tfidf2.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [64]:
matrix_tf1=X_train_tfidf2.toarray()
c=list(df2_all_n.columns)
merged_last = pd.DataFrame(matrix_tf1,columns=c)
merged_last['ID']=idd
merged_last['Parent_node_1']=Parent_node_1
merged_last['Parent_node_2']=Parent_node_2
merged_last['Leaf_Node']=leaf_node
merged_last['price_x']=prince

In [65]:
merged_last.head()

,360fly_x,6Ave,Air Hogs,AmazingForLess,Axess Products,Bell+Howell,BlackMagic,Blackmagic,Blackmagic Design,CANON,...,write,yaw,yield,zone,zoom_y,ID,Parent_node_1,Parent_node_2,Leaf_Node,price_x
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,bestbuy-13fb43ddbd8d9cef68833c1b9dcc8030,Cam Corders,Action Cameras,360 Degree Cameras,499.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,bestbuy-1a4c07203f6dfa7a994b27a7f8dd8d0a,Photo Devices,DSLR,DSLR,3699.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,bestbuy-1e1dc143119f5e0c4deaa9d7ac3f9583,Photo Devices,Mirrorless Cameras,NonFullFrameMirrorlessCamera,899.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,bestbuy-1e7969c673a012084c9f9bbe0e9794c0,Photo Devices,Mirrorless Cameras,NonFullFrameMirrorlessCamera,599.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,bestbuy-239b2bed55930613da27e4c3f077b94e,Photo Devices,DSLR,DSLR,2099.0


In [66]:
#drop missing valued columns
model_html=merged_last.copy()

## Modeling

In [67]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sbn
import math

In [68]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report
    
def analytics(y_true, y_pred):#target of the test data #predictions as 0,1 #model (knnclassifier) #data_test
    print("Confusion Matrix:")
    print(confusion_matrix_report(y_true, y_pred))
    print("Accuracy Score:")
    print(accuracy_score(y_true, y_pred))
    print()
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    #print("AUC Score:")
    #print(AUC(target_test, model, data_test))

### Level1

In [69]:
model_html.shape

(335, 65294)

In [70]:
X_level1=model_html.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node'],axis=1)
y_level1=model_html['Parent_node_1']

In [71]:
#F_CLASSIF
feature_names = list(X_level1.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level1, y_level1)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [  9  18  27  31  33  42  43  45  48  49  58  63  70  82  84 103 130 145
 147 160 174 183 195 199 219 224 225 233 242 260 269 286 290 291 315 359
 387 401 416 431 447 478 482 485 496 502 516 520 543 554 556 558 562 574
 592 599 607 610 616 624 644 646 652 696 701 707 737 745 772 789 804 805
 813 814 861 863 864 872 878 930 933 944 945 946 947] are constant.

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning:

invalid value encountered in divide



In [72]:
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
X_level1=X_level1[new_features_f]

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level1, y_level1)
print(clf.best_params_)

{'penalty': 'l2', 'C': 1000}


In [73]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l2', C= 1000)
target_prediction = cross_val_predict(logreg, X_level1, y_level1, cv=cv)

In [74]:
analytics(y_level1,target_prediction) #real value vs. predicted values

Confusion Matrix:
              ______________Prediction_______________
               Cam Corders    Not Camera Photo Devices
  Cam Corders           22             0            76
   Not Camera            5             9             5
Photo Devices            7             0           211

Accuracy Score:
0.722388059701
()
Classification Report:
               precision    recall  f1-score   support

  Cam Corders       0.65      0.22      0.33        98
   Not Camera       1.00      0.47      0.64        19
Photo Devices       0.72      0.97      0.83       218

  avg / total       0.72      0.72      0.67       335



In [75]:
model_html['prediction_level1']=target_prediction

### Level2

In [76]:
level2_train=model_html[model_html['Parent_node_1']=='Cam Corders'] #this includes wrongly assigned instances

In [77]:
X_level2=level2_train.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1'],axis=1)
y_level2=level2_train['Parent_node_2']

In [78]:
#F_CLASSIF
feature_names = list(X_level2.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level2, y_level2)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    1     5     6 ..., 65284 65287 65288] are constant.



In [79]:
new_features_f

[u'Canon',
 u'tripod_x',
 u'rotation_x',
 u'ie090_x',
 u'tonersclickthroughtypeurlvaluebrowsesports',
 u'producttake',
 u'16317da81e51dd02a4ce20',
 u'f9978cbddaf8v1jpgurltpcompact',
 u'1data_image_index2',
 u'30mmadd',
 u'trackingadvanced',
 u'2f180669360twitterdescriptionbuy',
 u'20look',
 u'lxwxh57',
 u'pointsthere',
 u'4f',
 u'20was',
 u'd810_y',
 u'servicesautomotive',
 u'smell',
 u'image4k',
 u'accuracydisplayvalueproperty_7values16',
 u'compliantpicture',
 u'collectionget',
 u'suppliestitleclassroom',
 u'8d06',
 u'120x',
 u'cameratable',
 u'h2cookie',
 u'aremoveclassdisplaynonetext',
 u'lowiso',
 u'bluebluepinkpurpleredwhiteshow',
 u'reviews3716',
 u'zoommax',
 u'000saving6368',
 u'lensesnot',
 u'20targets',
 u'18200',
 u'speed45',
 u'2linktextshop',
 u'lcdcolor',
 u'printsalexsee',
 u'monitormonitor75',
 u'fs14140k_k5',
 u'operabilityall',
 u'628787fc',
 u'hdhdtv',
 u'1xusb',
 u'directorysourcebooks',
 u'22214mm',
 u'5valuesitem_idvalues55485021displaynameitem',
 u'nullelseretur

In [80]:
X_level2=X_level2[new_features_f]

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level2, y_level2)
print(clf.best_params_)

{'penalty': 'l1', 'C': 100}


In [81]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 100)
target_prediction = cross_val_predict(logreg, X_level2, y_level2, cv=cv)

In [82]:
analytics(y_level2,target_prediction) #real value vs. predicted values

Confusion Matrix:
                          ____________________Prediction____________________
                                    Action Cameras Traditional Video Cameras
           Action Cameras                       45                         4
Traditional Video Cameras                        7                        42

Accuracy Score:
0.887755102041
()
Classification Report:
                           precision    recall  f1-score   support

           Action Cameras       0.87      0.92      0.89        49
Traditional Video Cameras       0.91      0.86      0.88        49

              avg / total       0.89      0.89      0.89        98



In [83]:
level2_train['prediction_level2']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



##### 1. Traditional Video Cameras

In [84]:
#prediction_level2_camcorders
level3_train=level2_train[level2_train['Parent_node_2']=='Traditional Video Cameras']

In [85]:
X_level3=level3_train.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1','prediction_level2'],axis=1)
y_level3=level3_train['Leaf_Node']

In [86]:
#F_CLASSIF
feature_names = list(X_level3.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level3, y_level3)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    0     1     2 ..., 65284 65287 65288] are constant.



In [87]:
X_level3=X_level3[new_features_f]

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level3, y_level3)
print(clf.best_params_)

{'penalty': 'l2', 'C': 1}


In [88]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l2', C= 1)
target_prediction = cross_val_predict(logreg, X_level3, y_level3, cv=cv)

In [89]:
analytics(y_level3,target_prediction)

Confusion Matrix:
                           _____________________Prediction_____________________
                            Common Use Video Cameras Professional Video Cameras
  Common Use Video Cameras                        12                          6
Professional Video Cameras                         3                         28

Accuracy Score:
0.816326530612
()
Classification Report:
                            precision    recall  f1-score   support

  Common Use Video Cameras       0.80      0.67      0.73        18
Professional Video Cameras       0.82      0.90      0.86        31

               avg / total       0.81      0.82      0.81        49



In [90]:
level3_train['prediction_level3']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



##### 2. Action Video Cameras

In [91]:
#prediction_level2_camcorders
level3_train_actioncameras=level2_train[level2_train['Parent_node_2']=='Action Cameras']

In [92]:
level3_train_actioncameras.head()

,360fly_x,6Ave,Air Hogs,AmazingForLess,Axess Products,Bell+Howell,BlackMagic,Blackmagic,Blackmagic Design,CANON,...,yield,zone,zoom_y,ID,Parent_node_1,Parent_node_2,Leaf_Node,price_x,prediction_level1,prediction_level2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,bestbuy-13fb43ddbd8d9cef68833c1b9dcc8030,Cam Corders,Action Cameras,360 Degree Cameras,499.0,Photo Devices,Action Cameras
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,bestbuy-39e5559aaa666c0b0a7fb9e41464d097,Cam Corders,Action Cameras,Non 360 Degree Cameras,699.0,Photo Devices,Action Cameras
54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,newegg-63393275702a1c4cd733de9d2a35c6d4,Cam Corders,Action Cameras,Non 360 Degree Cameras,79.0,Cam Corders,Action Cameras
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,newegg-6227d61dada36ede61b89ede95eb6dc4,Cam Corders,Action Cameras,Non 360 Degree Cameras,130.0,Photo Devices,Action Cameras
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,newegg-64b494d9b9b5c685041ae5c147f4a2cf,Cam Corders,Action Cameras,Non 360 Degree Cameras,95.0,Photo Devices,Action Cameras


In [93]:
level3_train_actioncameras.shape

(49, 65296)

In [94]:
X_level3_ac=level3_train_actioncameras.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1','prediction_level2'],axis=1)
y_level3_ac=level3_train_actioncameras['Leaf_Node']

In [95]:
#F_CLASSIF
feature_names = list(X_level3_ac.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level3_ac, y_level3_ac)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    1     5     6 ..., 65286 65287 65288] are constant.



In [96]:
X_level3_ac=X_level3_ac[new_features_f]
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level3_ac, y_level3_ac)
print(clf.best_params_)

{'penalty': 'l1', 'C': 100}


In [97]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 100)
target_prediction = cross_val_predict(logreg, X_level3_ac, y_level3_ac, cv=cv)

In [98]:
analytics(y_level3_ac,target_prediction)

Confusion Matrix:
                       ____________________________Prediction____________________________
                          360 Degree Cameras                  Drone Non 360 Degree Cameras
    360 Degree Cameras                    10                      1                      5
                 Drone                     0                     12                      2
Non 360 Degree Cameras                     0                      0                     19

Accuracy Score:
0.836734693878
()
Classification Report:
                        precision    recall  f1-score   support

    360 Degree Cameras       1.00      0.62      0.77        16
                 Drone       0.92      0.86      0.89        14
Non 360 Degree Cameras       0.73      1.00      0.84        19

           avg / total       0.87      0.84      0.83        49



In [99]:
level3_train_actioncameras['prediction_level3']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Cameras

In [100]:
level2_train_cam=model_html[model_html['Parent_node_1']=='Photo Devices'] #this includes wrongly assigned instances

In [101]:
X_level2_cam=level2_train_cam.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1'],axis=1)
y_level2_cam=level2_train_cam['Parent_node_2']

In [102]:
#F_CLASSIF
feature_names = list(X_level2_cam.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level2_cam, y_level2_cam)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    0     2     3 ..., 65281 65285 65286] are constant.



In [103]:
X_level2_cam=X_level2_cam[new_features_f]
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level2_cam, y_level2_cam)
print(clf.best_params_)

{'penalty': 'l1', 'C': 100}


In [104]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 100)
target_prediction = cross_val_predict(logreg, X_level2_cam, y_level2_cam, cv=cv)

In [105]:
analytics(y_level2_cam,target_prediction)

Confusion Matrix:
                      __________________________Prediction___________________________
                                      DSLR    Mirrorless Cameras Point & Shoot Cameras
                 DSLR                   39                    10                     4
   Mirrorless Cameras                    9                    44                    11
Point & Shoot Cameras                    0                     5                    96

Accuracy Score:
0.821100917431
()
Classification Report:
                       precision    recall  f1-score   support

                 DSLR       0.81      0.74      0.77        53
   Mirrorless Cameras       0.75      0.69      0.72        64
Point & Shoot Cameras       0.86      0.95      0.91       101

          avg / total       0.82      0.82      0.82       218



In [106]:
level2_train_cam['prediction_level2']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



#### 1. Point & Shoot Cameras

In [107]:
#prediction_level2_camcorders
level3_train_pocam=level2_train_cam[level2_train_cam['Parent_node_2']=='Point & Shoot Cameras']

In [108]:
level3_train_pocam.shape

(101, 65296)

In [109]:
X_level3_po=level3_train_pocam.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1','prediction_level2'],axis=1)
y_level3_po=level3_train_pocam['Leaf_Node']

In [110]:
#F_CLASSIF
feature_names = list(X_level3_po.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level3_po, y_level3_po)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    0     1     2 ..., 65282 65285 65286] are constant.



In [111]:
X_level3_po=X_level3_po[new_features_f]
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level3_po, y_level3_po)
print(clf.best_params_)

{'penalty': 'l1', 'C': 10}


In [112]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 10)
target_prediction = cross_val_predict(logreg, X_level3_po, y_level3_po, cv=cv)

In [113]:
analytics(y_level3_po,target_prediction)

Confusion Matrix:
                        _____________________________Prediction______________________________
                                Bridge Cameras Digital Compact Cameras                 Instant
         Bridge Cameras                     22                      23                       0
Digital Compact Cameras                      4                      38                       0
                Instant                      0                       6                       8

Accuracy Score:
0.673267326733
()
Classification Report:
                         precision    recall  f1-score   support

         Bridge Cameras       0.85      0.49      0.62        45
Digital Compact Cameras       0.57      0.90      0.70        42
                Instant       1.00      0.57      0.73        14

            avg / total       0.75      0.67      0.67       101



In [114]:
level3_train_pocam['prediction_level3']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



#### 2. Mirrorless Cameras

In [115]:
#prediction_level2_camcorders
level3_train_miror=level2_train_cam[level2_train_cam['Parent_node_2']=='Mirrorless Cameras']

In [116]:
X_level3_mi=level3_train_miror.drop(['ID','Parent_node_1','Parent_node_2','Leaf_Node','prediction_level1','prediction_level2'],axis=1)
y_level3_mi=level3_train_miror['Leaf_Node']

In [117]:
#F_CLASSIF
feature_names = list(X_level3_mi.columns.values)
X_new = SelectKBest(f_classif, k=100).fit(X_level3_mi, y_level3_mi)
mask = X_new.get_support() #list of booleans
new_features_f = [] # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_f.append(feature)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning:

Features [    0     2     3 ..., 65285 65286 65287] are constant.



In [118]:
X_level3_mi=X_level3_mi[new_features_f]
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X_level3_mi, y_level3_mi)
print(clf.best_params_)

{'penalty': 'l1', 'C': 1000}


In [119]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l2', C= 100)
target_prediction = cross_val_predict(logreg, X_level3_mi, y_level3_mi, cv=cv)

In [120]:
analytics(y_level3_mi,target_prediction)

Confusion Matrix:
                             _______________________Prediction_______________________
                            Full Frame Mirrorless Camera NonFullFrameMirrorlessCamera
Full Frame Mirrorless Camera                          15                            8
NonFullFrameMirrorlessCamera                           2                           39

Accuracy Score:
0.84375
()
Classification Report:
                              precision    recall  f1-score   support

Full Frame Mirrorless Camera       0.88      0.65      0.75        23
NonFullFrameMirrorlessCamera       0.83      0.95      0.89        41

                 avg / total       0.85      0.84      0.84        64



In [121]:
level3_train_miror['prediction_level3']=target_prediction

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### COMBINING ALL

In [122]:
level3_dslr=level2_train_cam[level2_train_cam['Parent_node_2']=='DSLR']
level3_dslr['prediction_level3']=level3_dslr['prediction_level2']

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [123]:
level3_nocamera=model_html[model_html['Parent_node_1']=='Not Camera']
level3_nocamera['prediction_level2']=level3_nocamera['prediction_level1']
level3_nocamera['prediction_level3']=level3_nocamera['prediction_level1']

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [124]:
all_concat=pd.concat([level3_dslr,level3_nocamera,level3_train_miror,level3_train_pocam,level3_train_actioncameras,level3_train])

In [125]:
all_concat.head()

,360fly_x,6Ave,Air Hogs,AmazingForLess,Axess Products,Bell+Howell,BlackMagic,Blackmagic,Blackmagic Design,CANON,...,zone,zoom_y,ID,Parent_node_1,Parent_node_2,Leaf_Node,price_x,prediction_level1,prediction_level2,prediction_level3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bestbuy-1a4c07203f6dfa7a994b27a7f8dd8d0a,Photo Devices,DSLR,DSLR,3699.0,Photo Devices,DSLR,DSLR
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bestbuy-239b2bed55930613da27e4c3f077b94e,Photo Devices,DSLR,DSLR,2099.0,Photo Devices,DSLR,DSLR
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bestbuy-247e7bdd47261a138d635807f7a54a79,Photo Devices,DSLR,DSLR,249.0,Photo Devices,Point & Shoot Cameras,Point & Shoot Cameras
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bestbuy-a7e8c737bca5ecd802fee9d3aa45fc7f,Photo Devices,DSLR,DSLR,1999.0,Photo Devices,DSLR,DSLR
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,bestbuy-42810b0b2667c9d208dae39ca087d2fb,Photo Devices,DSLR,DSLR,2099.0,Photo Devices,DSLR,DSLR


In [126]:
all_concat.shape

(335, 65297)

In [127]:
analytics(all_concat['Parent_node_1'],all_concat['prediction_level1'])#y_level3_mi,target_prediction

Confusion Matrix:
              ______________Prediction_______________
               Cam Corders    Not Camera Photo Devices
  Cam Corders           22             0            76
   Not Camera            5             9             5
Photo Devices            7             0           211

Accuracy Score:
0.722388059701
()
Classification Report:
               precision    recall  f1-score   support

  Cam Corders       0.65      0.22      0.33        98
   Not Camera       1.00      0.47      0.64        19
Photo Devices       0.72      0.97      0.83       218

  avg / total       0.72      0.72      0.67       335



In [128]:
analytics(all_concat['Parent_node_2'],all_concat['prediction_level2'])#y_level3_mi,target_prediction

Confusion Matrix:
                          _______________________________________________________________________________________________Prediction_______________________________________________________________________________________________
                                    Action Cameras               Cam Corders                      DSLR        Mirrorless Cameras                Not Camera             Photo Devices     Point & Shoot Cameras Traditional Video Cameras
           Action Cameras                       45                         0                         0                         0                         0                         0                         0                         4
              Cam Corders                        0                         0                         0                         0                         0                         0                         0                         0
                     DSLR                        0      

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [129]:
analytics(all_concat['Leaf_Node'],all_concat['prediction_level3'])#y_level3_mi,target_prediction

Confusion Matrix:
                             ___________________________________________________________________________________________________________________________________________________________________________________________________________________________Prediction___________________________________________________________________________________________________________________________________________________________________________________________________________________________
                                      360 Degree Cameras               Bridge Cameras                  Cam Corders     Common Use Video Cameras                         DSLR      Digital Compact Cameras                        Drone Full Frame Mirrorless Camera                      Instant           Mirrorless Cameras       Non 360 Degree Cameras NonFullFrameMirrorlessCamera                   Not Camera                Photo Devices        Point & Shoot Cameras   Professional Video Cameras
          36